In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_soom
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import MinMaxScaler
import time

In [2]:
device = 'cpu'

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device='cpu'):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)
        
        return x


In [4]:
X, y = load_diabetes(return_X_y = True, scaled=False)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))

torch_data = TensorDataset(torch.Tensor(X).to(device), torch.Tensor(y).to(device))
data_loader = DataLoader(torch_data, batch_size=1000)

In [5]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.LM(model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=False, c1=1e-4, tau=0.1, line_search_method='backtrack')

times = []

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    start = time.perf_counter()
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)
        opt.update(loss)

        all_loss[epoch] += loss
    end = time.perf_counter()
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().cpu().numpy().item()))
    # print(', time spent {}'.format(end-start))
    times.append(end-start)

    if patience <= 0:
        break

print(sum(times)/len(times))

epoch:  0, loss: 0.10293138772249222
epoch:  1, loss: 0.09468388557434082
epoch:  2, loss: 0.08654583245515823
epoch:  3, loss: 0.07858307659626007
epoch:  4, loss: 0.0778888612985611
epoch:  5, loss: 0.073574960231781
epoch:  6, loss: 0.06216760352253914
epoch:  7, loss: 0.05486954376101494
epoch:  8, loss: 0.048601049929857254
epoch:  9, loss: 0.04106307402253151
epoch:  10, loss: 0.03831474483013153
epoch:  11, loss: 0.03490950167179108
epoch:  12, loss: 0.03080679476261139
epoch:  13, loss: 0.02905493788421154
epoch:  14, loss: 0.028696918860077858
epoch:  15, loss: 0.026741649955511093
epoch:  16, loss: 0.024706117808818817
epoch:  17, loss: 0.02330246940255165
epoch:  18, loss: 0.021804850548505783
epoch:  19, loss: 0.020625069737434387
epoch:  20, loss: 0.019669316709041595
epoch:  21, loss: 0.01820002682507038
epoch:  22, loss: 0.017799872905015945
epoch:  23, loss: 0.01752820611000061
epoch:  24, loss: 0.017249366268515587
epoch:  25, loss: 0.01700097881257534
epoch:  26, loss

In [6]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.LM(model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=True, c1=1e-4, tau=0.1, line_search_method='backtrack')

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)
        opt.update(loss)

        all_loss[epoch] += loss
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.27584630250930786
epoch:  1, loss: 0.258060485124588
epoch:  2, loss: 0.24565142393112183
epoch:  3, loss: 0.22019362449645996
epoch:  4, loss: 0.19657878577709198
epoch:  5, loss: 0.17744438350200653
epoch:  6, loss: 0.15840564668178558
epoch:  7, loss: 0.14702625572681427
epoch:  8, loss: 0.13465726375579834
epoch:  9, loss: 0.12330587208271027
epoch:  10, loss: 0.11314216256141663
epoch:  11, loss: 0.10373230278491974
epoch:  12, loss: 0.09607663005590439
epoch:  13, loss: 0.08806784451007843
epoch:  14, loss: 0.0817742869257927
epoch:  15, loss: 0.07486937940120697
epoch:  16, loss: 0.068750761449337
epoch:  17, loss: 0.063807912170887
epoch:  18, loss: 0.05851135402917862
epoch:  19, loss: 0.053602393716573715
epoch:  20, loss: 0.04966318607330322
epoch:  21, loss: 0.045875340700149536
epoch:  22, loss: 0.04324166104197502
epoch:  23, loss: 0.040094904601573944
epoch:  24, loss: 0.03666651248931885
epoch:  25, loss: 0.03529784828424454
epoch:  26, loss: 0.032704